In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Smoothie_King/adjust_for_pending_stores'

In [2]:
writer_folder="./output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [3]:
excel_output=pd.ExcelFile("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190930/SmoothieKing_TA_of_3_miles_zips_JL_2019-10-03.xlsx")
excel_output.sheet_names


['summary_by_TA', 'output_TA_by_store', 'zip_TA', 'summary_by_store_count']

In [4]:
df_by_store=excel_output.parse("output_TA_by_store",dtype=str)
df_by_store.shape

(938, 18)

In [5]:
new_store_excel=pd.ExcelFile("/home/jian/Projects/Smoothie_King/SK_celery/force_store_list/SK_forced_store_list_20191002.xlsx")
print(new_store_excel.sheet_names)
df_new_stores=new_store_excel.parse("opening",dtype=str)
new_store_list=df_new_stores['storenumber'].unique().tolist()
print(len(new_store_list))
print(new_store_list)

['opening', 'closed']
5
['1805', '1727', '1735', '1787', '1549']


In [6]:
df_by_store[df_by_store['storenumber'].isin(new_store_list)]

# All of the new stores need to be added

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA


In [7]:
df_new_stores

,storenumber
0,1805
1,1727
2,1735
3,1787
4,1549


In [8]:
TA_name=df_by_store[['storenumber','TA']].drop_duplicates()
TA_name_count=TA_name.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
TA_name_list=TA_name.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
TA_name=pd.merge(TA_name_count,TA_name_list,on="TA",how="outer")
TA_name['store_count']=TA_name['store_count'].apply(lambda x: str(x))
TA_name['store_list']=TA_name['store_list'].apply(lambda x: str(x))
TA_name['TA_new_name']=TA_name['store_count']+"_"+TA_name['store_list']
TA_name_dict=TA_name.set_index("TA").to_dict()['TA_new_name']


df_by_store['TA']=df_by_store['TA'].apply(lambda x: TA_name_dict[x])
df_by_store=df_by_store.rename(columns={"TA":"TA_new"})


In [9]:
df_by_store.head(2)

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA_new
0,0000,7215 Veterans Memorial Blvd.,Metairie,LA,70003,1973-07-07 00:00:00,30.00589291,-90.22051644,Franchised,Steve Shields,70.183.114.37,Y,Same,nan,30.00589291,-90.22051644,"['70062', '70141', '70006', '70065', '70003', ...","15_['0000', '0002', '0003', '0006', '0010', '0..."
1,0002,4539 Veterans Blvd.,Metairie,LA,70006,1989-09-11 00:00:00,30.00581824,-90.18467266,Franchised,Steve Shields,108.65.143.116,N,Same,nan,30.00581824,-90.18467266,"['70009', '70006', '70001', '70183', '70002', ...","15_['0000', '0002', '0003', '0006', '0010', '0..."


In [10]:
zip_TA=excel_output.parse("zip_TA",dtype=str)
zip_TA['TA']=zip_TA['TA'].apply(lambda x: TA_name_dict[x])
zip_TA=zip_TA.rename(columns={"TA":"TA_new"})

# The automatic run region is not updated
del zip_TA['Co-Op/Regional']

In [11]:
region_by_store_file=pd.ExcelFile("/home/jian/Projects/Smoothie_King/adjust_for_pending_stores/region_mapping/Updated RAF List - Number of Opened Stores-2019-09-13.xlsx")
mapping_store_region=region_by_store_file.parse('All Stores',dtype=str)
mapping_store_region=mapping_store_region[['Store: Store Number  ↑','Co-Op/Regional  ↑']]
mapping_store_region.columns=['store','Co-Op/Regional']
mapping_store_region['store']=mapping_store_region['store'].apply(lambda x: x.replace("SK-",""))
print(mapping_store_region['store'].apply(lambda x: len(x)).unique())

zip_TA=pd.merge(zip_TA,mapping_store_region,on="store",how="left")
zip_TA['Co-Op/Regional']=zip_TA['Co-Op/Regional'].fillna("Unknown")

[4]


In [12]:
dict_new_stores={}

# 1805	4986 Steiner Way Suite 3 Grovetown, GA 30813
dict_new_stores.update({"1805":{"GoogleAddress":"4986 Steiner Way #3, Grovetown, GA 30813","Google_LatLng":[33.4806824,-82.1987107]}})

# 1727	301 Central Avuenue, Unit 7 Clark, NJ 07066
dict_new_stores.update({"1727":{"GoogleAddress":"301 Central Ave #7, Clark, NJ 07066","Google_LatLng":[40.6317773,-74.3137618]}})

# 1735	10225 Ocean Highway Suite 601 Pawleys Island, SC 29585
dict_new_stores.update({"1735":{"GoogleAddress":"10225 Ocean Hwy #601, Pawleys Island, SC 29585","Google_LatLng":[33.4378257,-79.1320184]}})

# 1787	317 Homer Adams Parkway, STE L Alton, IL 62002
dict_new_stores.update({"1787":{"GoogleAddress":"317 Homer Adams Pkwy STE L, Alton, IL 62002","Google_LatLng":[38.9189721,-90.1711105]}})

# 1549	2426 SE Military, Building 700 San Antonio, TX 78223
dict_new_stores.update({"1549":{"GoogleAddress":"2426 SE Military Dr Building 700, San Antonio, TX 78223","Google_LatLng":[29.3517249,-98.4541786]}})



In [14]:
df_new_stores['GoogleAddress']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['GoogleAddress'])
df_new_stores['GoogleLatitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][0])
df_new_stores['GoogleLongitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][1])


In [15]:
df_new_stores['zips_3_miles']=np.nan

for ind,row in df_new_stores.iterrows():
    store_center=(row['GoogleLatitude'],row['GoogleLongitude'])
    zip_list=[]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=3:
            zip_list.extend([zip_cd])
    df_new_stores.loc[ind,'zips_3_miles']=str(zip_list)


In [16]:
df_new_stores

,storenumber,GoogleAddress,GoogleLatitude,GoogleLongitude,zips_3_miles
0,1805,"4986 Steiner Way #3, Grovetown, GA 30813",33.480682,-82.198711,['30813']
1,1727,"301 Central Ave #7, Clark, NJ 07066",40.631777,-74.313762,"['07027', '07033', '07066', '07067', '07065', ..."
2,1735,"10225 Ocean Hwy #601, Pawleys Island, SC 29585",33.437826,-79.132018,['29585']
3,1787,"317 Homer Adams Pkwy STE L, Alton, IL 62002",38.918972,-90.171110,['62002']
4,1549,"2426 SE Military Dr Building 700, San Antonio,...",29.351725,-98.454179,"['78235', '78214', '78223']"


In [17]:
cols=zip_TA.columns.tolist()
zip_ta_new_stores=pd.DataFrame(columns=zip_TA.columns.tolist())
for ind, row in df_new_stores.iterrows():
    store=row['storenumber']
    zip_list=eval(row['zips_3_miles'])
    df=pd.DataFrame({"zip":zip_list},index=[store]*len(zip_list)).reset_index().rename(columns={"index":"store"})
    df['TA_new']="new_stores"
    df['Primary_DMA']="new_stores"
    df['Co-Op/Regional']="Unknown"
    zip_ta_new_stores=zip_ta_new_stores.append(df)
zip_ta_new_stores=zip_ta_new_stores[cols]

In [18]:
zip_TA=zip_TA.append(zip_ta_new_stores)
df_TA_zips=zip_TA.copy()

In [19]:
for region,df_group in df_TA_zips.groupby("Co-Op/Regional"):
    if region!="UnknowRegion":
        df_group=df_group[['zip','Co-Op/Regional']].drop_duplicates()
        df_group.to_csv(writer_folder+"zips_for_the_%s.csv"%region,index=False)
    else:
        # df_austin=df_group[df_group['store'].isin(Austin_stores['store'].tolist())]
        df_unknown=df_group[~df_group['store'].isin(Austin_stores['store'].tolist())]

        # df_austin.to_csv(writer_folder+"zips_for_the_%s.csv"%"AustinDMA",index=False)
        df_unknown.to_csv(writer_folder+"zips_for_the_%s.csv"%"UnknownRegion",index=False)
df_output_overall_zips=df_TA_zips[['zip']].drop_duplicates()
df_output_overall_zips.to_csv(writer_folder+"overall_zips_in_TA.csv",index=False)

In [20]:
writer=pd.ExcelWriter(writer_folder+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_TA_zips.to_excel(writer,"df_TA_zips",index=False)
df_new_stores.to_excel(writer,"new_stores_add_on",index=False)
df_by_store.to_excel(writer,"df_by_store",index=False)
rfa_mapping=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/RFA_region_20190128.xlsx",dtype=str)
rfa_mapping.to_excel(writer,"RFA_region_20190128",index=False)
writer.save()